In [94]:
import os 
import numpy as np 
from sklearn.model_selection  import train_test_split

In [95]:
datafile = open(os.getcwd() + '/spambase/spambase.data' , 'r')

In [96]:
data = [] ;
for line in datafile:
    line = [float(element)for element in line.rstrip('\n').split(',')]
    data.append(np.asarray(line))

In [97]:
num_features = 48 ;
x = [ data[i][:num_features] for i in range(len(data))]
y = [ int( data[i][-1]) for i in range(len(data))]


In [98]:
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size=0.25)

In [99]:
# making likelihood estimation

# find the two classes 
x_train_class_0 = [ x_train[i] for i in range( len(x_train)) if y_train[i]==0]
x_train_class_1 = [ x_train[i] for i in range(len(x_train)) if y_train[i]==1]

#find the class specific likelihood of each features
likelihoods_class_0 = np.mean(x_train_class_0 , axis = 0)/100.0
likelihoods_class_1 = np.mean( x_train_class_1, axis = 0)/100.0
                            
        

In [100]:
def cal_log_likelihood_nave_bayes(feature_vector, class_):
    assert len(feature_vector)== num_features
    log_likelihood = 0.0
    if class_ == 0:
        for feature_index in range(len(feature_vector)):
            if feature_vector[feature_index] == 1: 
                log_likelihood += np.log10(likelihoods_class_0[feature_index])
            elif feature_vector[feature_index]== 0:
                log_likelihood += np.log10(1.0 - likelihoods_class_0[feature_index])
    elif class_ == 1:
        for feature_index in range(len(feature_vector)):
            if feature_vector[feature_index]== 1:
                log_likelihood += np.log10(likelihoods_class_1[feature_index])
            elif feature_vector[feature_index]==0:
                log_likelihood += np.log10(1-likelihoods_class_1[feature_index])
    else :
        raise ValueError('class takes integer val 0 or 1')
    return log_likelihood

In [101]:
# calculating the class priors 
num_class_0 = float( len(x_train_class_0))
num_class_1 = float(len(x_train_class_1))

prior_probability_class_0 = num_class_0/(num_class_0 + num_class_1)
prior_probability_class_1 = num_class_1/(num_class_0 + num_class_1)

log_prior_class_0 = np.log10(prior_probability_class_0)
log_prior_class_1= np.log10(prior_probability_class_1)

In [102]:
def cal_class_posteriors(feature_vector):
    log_likelihood_class_0 = cal_log_likelihood_nave_bayes( feature_vector,class_=0)
    log_likelihood_class_1 = cal_log_likelihood_nave_bayes(feature_vector,class_=1)
    
    log_posterior_class_0 = log_likelihood_class_0 + log_prior_class_0
    log_posterior_class_1= log_likelihood_class_1+ log_prior_class_1
    return log_posterior_class_0 , log_posterior_class_1

In [103]:
def classify_spam(document_vector):
    feature_vector = [int(element>0) for element in document_vector]
    log_posterior_class_0 , log_posterior_class_1 = cal_class_posteriors(feature_vector)
    if log_posterior_class_0 > log_posterior_class_1:
        return 0 
    else:
        return 1
        

In [104]:
# predction spam or not on the test data
prediction = [];
for email in x_test:
    prediction.append(classify_spam(email))

def evaluate_performance(prediction , ground_truth_labels):
    correct_count = 0.0;
    for item_idx in range(len(prediction)):
        if prediction[item_idx] == ground_truth_labels[item_idx]:
            correct_count += 1.0
    accuracy = correct_count/len(prediction)
    return accuracy 

In [105]:
acc_of_naive_bayes = evaluate_performance(prediction , y_test)
print(acc_of_naive_bayes)

0.8601216333622936
